In [2]:
import pandas as pd
from tqdm.auto import tqdm
import random
from rdkit import Chem
import numpy as np
from collections import defaultdict
import pyarrow.parquet as pq
import pandas as pd
from pymongo import MongoClient

/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = pd.read_parquet('test.parquet')['id']
len(ds)

1674896

In [15]:
pd.DataFrame(ds.molecule_smiles.unique(), columns=['molecule_smiles']).to_parquet('molecule_smiles_test.parquet')

,molecule_smiles
0,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...
1,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2...
2,C#CCCC[C@H](Nc1nc(NCC2(O)CCCC2(C)C)nc(Nc2ccc(C...
3,C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2sc(Cl)c...
4,C#CCCC[C@H](Nc1nc(NCC2CCC(SC)CC2)nc(Nc2ccc(C=C...
...,...
878017,Cn1ncc2cc(Nc3nc(Nc4nncs4)nc(N[C@@H](CCCN=[N+]=...
878018,[N-]=[N+]=NCCC[C@H](Nc1nc(NCC2CCC3CC3C2)nc(Nc2...
878019,COC(=O)c1ccnc(Nc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@...
878020,COC1CCC(CCNc2nc(Nc3noc4ccc(F)cc34)nc(N[C@@H](C...


In [4]:
from pymongo import MongoClient

# Connect to MongoDB server
client = MongoClient('mongodb://localhost:27017/')  # Adjust the connection string as needed

# Select database and collection
db = client['belka']  # Replace with your database name
collection = db['full_molecules']  # Replace with your collection name

pipeline = [
    {'query' : {'id': }}
    {'$group': {
        '_id': '$field_name',  # Field to check for duplicates
        'uniqueIds': {'$addToSet': '$_id'},
        'count': {'$sum': 1}
    }},
    {'$match': {
        'count': {'$gt': 1}  # Condition to identify duplicates
    }}
]

duplicates = collection.aggregate(pipeline, allowDiskUse=True)


OperationFailure: PlanExecutor error during aggregation :: caused by :: Used too much memory for a single array. Memory limit: 104857600. Current set has 4993220 elements and is 104857620 bytes., full error: {'ok': 0.0, 'errmsg': 'PlanExecutor error during aggregation :: caused by :: Used too much memory for a single array. Memory limit: 104857600. Current set has 4993220 elements and is 104857620 bytes.', 'code': 146, 'codeName': 'ExceededMemoryLimit'}

In [ ]:
# Remove duplicates
for document in duplicates:
    ids_to_delete = document['uniqueIds'][1:]  # Skip the first id to keep one entry
    collection.delete_many({'_id': {'$in': ids_to_delete}})